In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd
additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

DATABASE_NAME = "cbr_database_15_30"
OUTPUT_FILE = "validation_cbr_15_30_gpt2_input"

IX_FILE = "cbr_augmentation_1_gpt2_results_p_ix.csv"
dfix = pd.read_csv(f"{IX_FILE}")
cbr_id = cbr_database.set_index("id")

In [2]:
cbr_db = pd.concat([cbr_database, additional_cbr_1])
cbr_db_ix = pd.concat([cbr_database, dfix])
cbr_db_ix = cbr_db_ix.set_index("id")

large_cbr_reps = str_rep_vdb(cbr_db)
len(large_cbr_reps)

15940

In [3]:
cbr_db_ix

,name,steps,ingredients
id,,,
74113,creamy swiss chicken breasts,Arrange chicken in a 13x9x2 inch baking dish t...,"chicken breasts, swiss cheese, low-fat cream o..."
257796,slow cooker garlic chicken with rosemary,"Place rosemary springs, 1 lemon half, celery, ...","roasting chickens, lemons, rosemary sprigs, pa..."
251645,curried chicken rice salad,"Combine cool rice, chicken, pineapple, celery ...","raw rice, cooked chicken, canned pineapple, ce..."
389514,omelet with corn and smoked mozzarella,Cut corn from cob or use frozen and place in a...,"fresh corn, water, eggs, salt & freshly ground..."
349283,chilled peach soup with riesling,"In a large saucepan, combine the riesling, clo...","riesling wine, whole cloves, cinnamon stick, b..."
...,...,...,...
432656,layered pineapple and lemon cheesecake pie,Preheat oven to 325 degrees. Beat cream cheese...,"cream cheese, granulated sugar, cool whip topp..."
120055,garlic parmesan mushrooms,Preheat oven to 375f. Grease a 13 x 9-inch bak...,"mushroom, reduced-calorie margarine, garlic sa..."
145151,candied popcorn splenda,"Combine sugar, molasses and molasses in suacep...","popped popcorn, egg white, dark molasses, vani..."


In [4]:
# large_cbr = pd.concat([cbr_database, cbr_augmentation, additional_cbr_1, additional_cbr_2])
# large_cbr_id = large_cbr.set_index("id")
# large_cbr_reps = str_rep_vdb(large_cbr_id)
# len(large_cbr_reps)

In [5]:
# meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db["id"].values)], {"dataset":"cbr_15_30"})
# vdb = CBRDatabase(large_cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)
print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=1))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
15940
[Document(page_content='name: mock pineapple ; ingredients: unsweetened pineapple juice, zucchini, sugar, lemon juice, food coloring ; preparation:', metadata={'dataset': 'cbr_15_30', 'recipe_id': 96738})]


In [6]:
list(range(4, -1, -1))

[4, 3, 2, 1, 0]

In [7]:
# generowanie inputów - 850 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        # print(tem)#dev
        # print(compose(cbr_db_ix.loc[[recipe_ixs[0]]], v_recipe))#dev
        # raise

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 710
1 - 727
1 - 663
1 - 734
1 - 675
1 - 723
1 - 767
2 - 646
1 - 765
1 - 738
1 - 540
1 - 817
1 - 846
1 - 706
1 - 668
1 - 664
1 - 705
1 - 842
1 - 846
1 - 838
1 - 715
1 - 832
1 - 773
1 - 721
1 - 787
1 - 841
1 - 722
1 - 718
1 - 636
1 - 746
1 - 768
1 - 743
1 - 753
1 - 809
1 - 807
1 - 736
1 - 778
1 - 687
1 - 848
1 - 839
1 - 783
1 - 745
1 - 717
1 - 843
1 - 669
1 - 709
1 - 768
1 - 841
1 - 711
1 - 817
1 - 822
1 - 807
1 - 806
1 - 701
1 - 749
1 - 731
2 - 646
1 - 791
1 - 848
1 - 771
1 - 725
1 - 711
2 - 840
1 - 772
1 - 653
2 - 690
1 - 688
1 - 806
1 - 783
1 - 701
1 - 721
1 - 766
1 - 689
1 - 779
1 - 743
1 - 572
1 - 688
1 - 702
1 - 724
1 - 792
1 - 735
1 - 782
1 - 764
1 - 705
1 - 847
1 - 843
1 - 708
1 - 769
1 - 834
2 - 717
1 - 762
1 - 832
1 - 832
1 - 722
1 - 681
1 - 826
1 - 750
1 - 752
1 - 810
1 - 717
1 - 768
1 - 772
1 - 812
1 - 762
1 - 826
2 - 716
1 - 684
1 - 591
1 - 844
1 - 690
1 - 797
1 - 766
1 - 793
1 - 749
1 - 756
1 - 731
1 - 743
1 - 784
1 - 787
1 - 695
1 - 625
1 - 811
1 - 695
1 - 723
1 - 719


Token indices sequence length is longer than the specified maximum sequence length for this model (2064 > 2048). Running this sequence through the model will result in indexing errors


2 - 433
1 - 719
1 - 786
1 - 839
1 - 733
1 - 819
1 - 768
1 - 734
1 - 685
1 - 846
1 - 767
2 - 724
1 - 636
2 - 722
1 - 786
1 - 573
1 - 749
1 - 693
1 - 710
1 - 617
1 - 822
1 - 782
1 - 733
1 - 675
1 - 713
1 - 732
1 - 597
1 - 673
1 - 782
1 - 687
1 - 843
1 - 768
2 - 654
1 - 779
1 - 849
1 - 711
1 - 704
1 - 755
1 - 693
1 - 732
1 - 823
1 - 680
1 - 821
1 - 706
1 - 782
1 - 653
1 - 658
1 - 761
1 - 722
1 - 747
1 - 816
1 - 755
1 - 754
1 - 714
1 - 683
1 - 686
1 - 668
1 - 778
1 - 707
1 - 763
1 - 763
1 - 688
1 - 724
1 - 668
1 - 827
1 - 720
1 - 690
1 - 674
1 - 687
1 - 772
2 - 588
1 - 777
1 - 726
1 - 725
1 - 741
1 - 773
1 - 702
1 - 710
1 - 737
1 - 700
1 - 747
1 - 781
2 - 662
1 - 811
1 - 770
1 - 757
1 - 673
1 - 763
1 - 757
1 - 789
1 - 690
1 - 729
1 - 652
1 - 714
1 - 773
1 - 711
1 - 499
2 - 804
1 - 783
1 - 674
1 - 803
1 - 782
1 - 616
1 - 835
1 - 721
1 - 732
1 - 722
1 - 712
1 - 708
1 - 787
1 - 682
1 - 612
1 - 810
1 - 747
1 - 681
1 - 745
1 - 673
1 - 671
1 - 840
1 - 800
1 - 736
1 - 839


In [8]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [9]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=7)

[Document(page_content='name: naked tater all the way ; ingredients: potatoes, butter, sour cream, milk, bacon bits, cheddar cheese, green onions, garlic powder, salt and pepper ; preparation:', metadata={'dataset': 'cbr_15_30', 'recipe_id': 135691}),
 Document(page_content='name: make ahead cheese strata ; ingredients: white bread, butter, sharp cheddar cheese, eggs, milk, salt, red pepper, dry mustard, worcestershire sauce, celery leaves ; preparation:', metadata={'dataset': 'cbr_15_30', 'recipe_id': 230217}),
 Document(page_content='name: hunter s delight ; ingredients: bacon, red potatoes, onions, venison steak, cream-style corn, worcestershire sauce, sugar, seasoning salt ; preparation:', metadata={'dataset': 'cbr_15_30', 'recipe_id': 331054}),
 Document(page_content='name: lemon roasted asparagus gratin ; ingredients: asparagus spears, butter, garlic clove, lemon zest, pepper, salt, fresh breadcrumb, swiss cheese, lemon juice ; preparation:', metadata={'dataset': 'cbr_15_30', 're

In [10]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1051, 7: 1036, 5: 37, 4: 1})